In [127]:
# RUN ONCE
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("marcozuppelli/stegoimagesdataset")
# print("Path to dataset files:", path)

In [128]:
from PIL import Image
import numpy as np
import pandas as pd
import glob
import random
from sklearn.utils import shuffle



In [129]:
def getLSBBytes(filePath):
    img = Image.open(filePath).convert("RGB")
    pixels = np.array(img).flatten() #convert to an np array and flatten it so that the rgb values are converted to a single array instead of three separate
    lsb_array = pixels & 1  # isolate the lsb values by anding with 1

    # Trim the array's length to a multiple of 8 to ensure we don't get errors in collecting raw bytes from the binary
    usable_length = (len(lsb_array) // 8) * 8
    lsb_array = lsb_array[:usable_length]
    #display(lsb_array)

    # Transform into a 2d array with each row being 8 bits so that we can then convert each row into a byte with numpy
    lsb_reshaped = lsb_array.reshape((-1, 8))
    #display(lsb_reshaped)

    # Convert each 8-bit array to a single byte
    powers = 2**np.arange(7, -1, -1)
    byte_values = np.dot(lsb_reshaped, powers) #uses np vectors to quickly transform the bits into a byte

    # Convert to actual bytes
    return byte_values.astype(np.uint8).tobytes() #converts the type to a byte


In [130]:
### GETTING THE PATHS OF ALL THE IMAGES IN THE KAGGLE DATASET
basePath = '/Users/gaim/.cache/kagglehub/datasets/marcozuppelli/stegoimagesdataset/versions/2/train/train'

stegImgs = glob.glob(basePath + '/stego/*.png') #get all the steg image paths
cleanImgs = glob.glob(basePath + '/clean/*.png') #get all the clean image paths



In [131]:
### GET THE RAW BYTES FOR EACH IMAGE AND THEN ADD THEM TO A PANDAS DATAFRAME

rows = []
for img in stegImgs:
    rowInfo = {
        'imagePath' : img,
        'rawBytes' : getLSBBytes(img),
        'hasSteg' : 1
    }
    rows.append(rowInfo)
    #print(img, "COMPLETED")


for img in cleanImgs:
    rowInfo = {
        'imagePath' : img,
        'rawBytes' : getLSBBytes(img),
        'hasSteg' : 0
    }
    rows.append(rowInfo)
    


df = pd.DataFrame(rows)
df = shuffle(df, random_state=8)
display(df)

,imagePath,rawBytes,hasSteg
5266,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00*X\xcc0ZAiL\x8c4Y\xc1`x\x95\xedI\xc1aF\x...,1
4571,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00\x00\x00\x00\x00\x00\xd5\x96e\x08\x82d\t...,1
4494,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00\x00\x00\x00\x00\x00y#K\xd5?\x0e\xd43o\x...,1
6022,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00\x00\x00\x00\x00\x00\xd7^%\x08\x82d\t\x0...,1
4350,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00/(\xd4u\x18\x8c\xaeo\xd5\xfd_\x98\xb9Z\x...,1
...,...,...,...
2181,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00\x00\x00\x00\x00\x00\x03f\tU\xb8}X\x95_\...,1
10601,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00\x00\x00\x00\x00\x00\x0b\x0eI\xc1\xa0=\x...,1
2033,/Users/gaim/.cache/kagglehub/datasets/marcozup...,b'\x00\x00\x00\x00\x00\x00\xcdhe\x08\x82d\t\x0...,1
9556,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"b'\x00\x00\x00\x00\x00\x04\x8d\x80""\x80 \x08\x...",1


In [137]:
#display(df['hasSteg'].value_counts())

In [87]:
# path = '/Users/gaim/.cache/kagglehub/datasets/marcozuppelli/stegoimagesdataset/versions/2'
# image = path+ '/train/train/stego/image_00329_js_1.png'

# # image = './assets/steg_image.png'

# img = Image.open(image).convert('RGB')

# pixels = np.array(img)

# display(pixels)

# pixelsFlattened = pixels.flatten()
# display(pixelsFlattened)

In [88]:
# lsbArray = pixelsFlattened & 1 #this will isolate the least significant bit as we 

# #Now we can analyze the array of least significant bits to get a string of bits
# groupedLSB = []
# binary = ""
# for idx in range (0, len(lsbArray)):
#     binary += str(lsbArray[idx])
#     if (idx % 8 == 7):
#         groupedLSB.append(binary)
#         binary = ""

# display(groupedLSB)

In [89]:
# byteArray = bytearray()
# for binString in groupedLSB:
#     if (len(binString) == 8):
#         byte = int(binString,2)
#         byteArray.append(byte)

# outputMessage = bytes(byteArray)

In [91]:
# def embed_message(image_path, message, output_path):
#     # Open and convert image
#     img = Image.open(image_path).convert("RGB")
#     pixels = np.array(img)
#     flat_pixels = pixels.flatten()

#     # Convert message to binary + add null terminator
#     message += '\0'  # Marks the end of the message
#     binary_message = ''.join([format(ord(c), '08b') for c in message])
    
#     # Check if the image can hold the message
#     if len(binary_message) > len(flat_pixels):
#         raise ValueError("Message is too long to fit in the image.")

#     # Embed each bit of the message
#     for i, bit in enumerate(binary_message):
#         flat_pixels[i] = (flat_pixels[i] & 1) | int(bit)  # Clear LSB and set it to bit

#     # Reshape and save the new image
#     new_pixels = flat_pixels.reshape(pixels.shape)
#     new_img = Image.fromarray(new_pixels.astype('uint8'), 'RGB')
#     new_img.save(output_path)
#     print(f"✅ Message embedded and saved to: {output_path}")

In [92]:
# embed_message(
#     image_path='./assets/garfield.png',          # your input image
#     message='hello from the stego world',  # your secret message
#     output_path='./assets/steg_image.png'          # image to save
# )